In [1]:
import os
import cv2
import matplotlib as plt
from PIL import Image

def removeHair(img_org, img_gray, kernel_size=25, threshold=10, radius=3):
    # kernel for the morphological filtering
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (kernel_size, kernel_size))

    # perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)

    # intensify the hair countours in preparation for the inpainting algorithm
    _, thresh = cv2.threshold(blackhat, threshold, 255, cv2.THRESH_BINARY)

    # inpaint the original image depending on the mask
    img_out = cv2.inpaint(img_org, thresh, radius, cv2.INPAINT_TELEA)

    processed_image = img_out

    return processed_image

# Define the folder containing images
folder_path = "data 1310-1410"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
        image_path = os.path.join(folder_path, filename)

        # Load image using OpenCV
        img_cv = cv2.imread(image_path)
        gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    
         # Remove hair from the grayscale image (pass any necessary parameter)
        processed_image = removeHair(img_cv, gray_cv)  # Replace 'your_value' with the actual argument for your function

        # Display the image without hair
        cv2.imshow(f"Processed - {filename}", processed_image)

        # Wait for a key press before closing the image window
        cv2.waitKey(0)
        cv2.destroyAllWindows()  # Close the window after key press
        

        print(f"Displayed: {filename}")

Displayed: img_1310.png
Displayed: img_1311.png
Displayed: img_1312.png
Displayed: img_1313.png
Displayed: img_1314.png
Displayed: img_1315.png
Displayed: img_1316.png
Displayed: img_1317.png
Displayed: img_1318.png
Displayed: img_1319.png
Displayed: img_1320.png
Displayed: img_1321.png
Displayed: img_1322.png
Displayed: img_1323.png
Displayed: img_1324.png
Displayed: img_1325.png
Displayed: img_1326.png
Displayed: img_1327.png
Displayed: img_1328.png
Displayed: img_1329.png
Displayed: img_1330.png
Displayed: img_1331.png
Displayed: img_1332.png
Displayed: img_1333.png
Displayed: img_1334.png
Displayed: img_1335.png
Displayed: img_1336.png
Displayed: img_1337.png
Displayed: img_1338.png
Displayed: img_1339.png
Displayed: img_1340.png
Displayed: img_1341.png
Displayed: img_1342.png
Displayed: img_1343.png
Displayed: img_1344.png
Displayed: img_1345.png
Displayed: img_1346.png
Displayed: img_1347.png
Displayed: img_1348.png
Displayed: img_1349.png
Displayed: img_1350.png
Displayed: img_1